In [1]:
#threshold_fixed
import random
import numpy as np
random.seed(100000)
np.random.seed(100000)
import scipy.stats as stats
from scipy.stats import pearsonr
from scipy.stats import norm

def gini_coefficient(x):
    """Compute Gini coefficient of array of values"""
    diffsum = 0
    for i, xi in enumerate(x[:-1], 1):
        diffsum += np.sum(np.abs(xi - x[i:]))
    return diffsum / (len(x)**2 * np.mean(x))

def f(a,N):
    """Get indices of the top N values of a list"""
    return np.argsort(a)[::-1][:N]

tmax= 1000
#-----------------------------
num = 600
nummax=600
#-----------------------------
reference = 20
refmax =100
#-----------------------------
reading = 120
readingmax=120
#-----------------------------
noise =0.05
fit =0.1
shape =6
list1=[] #top papers cited substantively
#list2=[] #top papers cited rhetorically
list3=[] #mid papers cited substantively
#list4=[] #mid papers cited rhetorically
#no rhetorical citing
listcorr=[]
listgini=[]

for num in range(num, nummax+1):
  normative=np.random.beta(1, shape, size=num)
  qrank =[]
  qrank = list(np.argsort(normative)[-(num):][::-1])
  weight=0.001
  for reference in range(reference, refmax+1):
  #for reading in range(reading, readingmax+1):

    top1q=qrank[0:40]
    top2q=qrank[40:int(150)]

    cite_population = [0]*num
    chunk =[]#churn

    for t in range(1, tmax+1):

      threshold =random.uniform(0,1)

      noise_list=np.random.normal(0, noise, num)

      fit_list=[]
      for i in range(num):
        fit_list.append(random.uniform(-fit, fit))

      pq =[]
      for i in range(num):
        pq.append(normative[i]+ fit_list[i] + noise_list[i])
      for i in range(num):
        if (pq[i] >1):
          pq[i] =1
        if (pq[i] <0):
          pq[i] =0

      signal =[]
      for i in range(num):
        signal.append(pq[i] + weight * cite_population[i] )
      for i in range(num):
        if (signal[i] >2):
          signal[i] =2
        if (signal[i] <0):
          signal[i] =0

      reading_index = list(np.argsort(signal)[-(reading):][::-1])

      norm_list =[]
      for i in range(len(reading_index)):
        norm_list.append(normative[reading_index[i]] + fit_list[reading_index[i]])

      for i in range(len(reading_index)):
        if (norm_list[i] <0):
          norm_list[i] =0
        if (norm_list[i] >1):
          norm_list[i] =1

      over_threshold=[]
      for i in range(len(norm_list)):
        if (norm_list[i]> threshold):
          over_threshold.append(reading_index[i])

#here is the major difference between this model and model 1.1
      #if there are enough good papers for substantive citing
      #cite the best ones within the citing budget
      #if there are insufficiently many good papers for substantive citing
      #then just cite those whose quality is above the substantive citing threshold
      normative_cite=[]
      overall_cite =[]
      if (len(over_threshold) >= reference):
        cite = list(f(norm_list, reference))
        for i in range(len(cite)):
          cite_population[reading_index[cite[i]]]= cite_population[reading_index[cite[i]]]+1
          normative_cite.append(reading_index[cite[i]])
        overall_cite = normative_cite.copy()

      else:
        normative_cite = over_threshold.copy()
        for i in range(len(normative_cite)):
          cite_population[normative_cite[i]] = cite_population[normative_cite[i]] +1
        overall_cite = normative_cite.copy()

      #churn
      chunk.append(overall_cite)

# SUB-ANALYSIS------------------------------------------------------------------
    #print(*cite_population, sep='\n')

    #listgini.append(gini_coefficient(np.array(cite_population)))

    corr1, _ = stats.pearsonr(normative, cite_population)
    listcorr.append(corr1)

    #print(corr1)
     # top1=0
     # for i in range(40):
     #   if ((top1q[i] in (normative_cite )) == True):
      #    top1 = top1+1
     # list1.append(top1/reference)
     # top3=0
     # for i in range(110):
     #   if ((top2q[i] in (normative_cite)) == True):
      #    top3 = top3+1
     # list3.append(top3/reference)

    #dnew=[]
   # for i in range(len(chunk) -1):
     # a=0
     # for j in range(len(chunk[i+1])):
     #   if ((chunk[i+1][j] in chunk[i]) == False):
      #    a=a+1
     # dnew.append(a)
   # print(np.mean(dnew))

#print(list1)
#print(list3)
#print(listgini)
#print(dnew)

print('example output: Correlation (citation-quality)')
print(*listcorr, sep='\n')
print('the null (threshold) model\'s correlation is lower than the full model')

example output: Correlation (citation-quality)
0.6565039121024128
0.6632395750327811
0.6723472307744774
0.6771553211297321
0.6833479888150037
0.6919292749529795
0.6970528815400677
0.702710178331455
0.71105471678854
0.7170255351191943
0.7226594704498973
0.7261106169911997
0.7321680503328499
0.7370028499960974
0.7438531252523719
0.749032932000787
0.7526500662686397
0.7584167740265526
0.7632640125038253
0.7666063011231119
0.7718649951084233
0.7752170313688859
0.7810545005844091
0.7815245386117148
0.784409225921704
0.7905413222611589
0.7928852591657759
0.7993962572728667
0.8024081502195123
0.8010648925707416
0.8092431408475751
0.811350176673205
0.8142645083981427
0.8193787318022839
0.8228494856559805
0.8242327684990594
0.8291920199534469
0.8312092843614217
0.831946651281711
0.8377789866117733
0.839168177675002
0.8405784047425884
0.8409128717321259
0.8465207602390368
0.8522246000455553
0.8517276790706043
0.8543310985687727
0.858086071837674
0.8595721185091901
0.8628141579408497
0.8628994377